In [1]:
import sqlite3
import pandas as pd

import sys
sys.path.append("../")
from controller import *

unable to import 'smart_open.gcs', disabling that module
[nltk_data] Downloading package stopwords to /home/andre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/andre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /home/andre/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
decretos_sen = load_senado_decrees('../Senado API files/', '../radar_db')

classes_sen=get_classes("../supervised NLP/Classification/classificacao senado")

decretos_classify_sen = multilabel_classify_decrees(decretos_sen, classes_sen)
# get_df_senado(decretos_classify_sen)

10135 is not classified by Senado.
10125 is not classified by Senado.
10021 is not classified by Senado.
9205 is not classified by Senado.
8420 is not classified by Senado.


In [3]:
decretos_minist = load_ministerio_decrees('../radar_db')

classes_minist=get_classes("../supervised NLP/Classification/classificacao referenda")

decretos_classify_minist = multilabel_classify_decrees(decretos_minist, classes_minist)
# get_df_ministerio(decretos_classify_minist)

In [4]:
classes_sen_names = [i[0] for i in classes_sen]
classes_sen_names = classes_sen_names + ['Não classificado']

classes_minist_names = [i[0] for i in classes_minist]
classes_minist_names = classes_minist_names + ['Não classificado']

In [5]:
decretos_minist_senado = pd.DataFrame()
decretos_minist_senado['cod_ident_ato'] = get_df_ministerio(decretos_classify_minist)['cod_ident_ato']
decretos_minist_senado['classificação senado'] = get_df_senado(decretos_classify_sen)['classificação senado']
decretos_minist_senado['referenda'] = get_df_ministerio(decretos_classify_minist)['referenda']

decretos_minist_senado.set_index('cod_ident_ato', inplace = True)
# decretos_minist_senado

In [6]:
# Gets some more data from the database to enrich analysis.
conn = sqlite3.connect('../radar_db')
c = conn.cursor()
c.execute("SELECT cod_ident_ato, ementa, chefe_de_governo, link_texto_original, referenda, links FROM Decretos")
decretos = c.fetchall()
conn.close()

decretos_df = pd.DataFrame(decretos)
decretos_df.set_index(0, inplace = True)

In [7]:
import csv
with open('graph csv/nodes_gephi_v3.csv', newline='') as csvfile:
    nodes_analysis = list(csv.reader(csvfile))
    
def to_names(serie, classes):
    names = ""
    for i in range(len(serie)):
        if(serie[i] == 1):
            names+=(classes[i] + " | ")
    return names

In [8]:
def print_modularity_analysis(number_modularity_class):
    print("Gephi modularity class", number_modularity_class, "\n")
    full_list_referenda = []
    full_list_sen = []
    counter = 0
    for n in nodes_analysis:
        if(n[6] == number_modularity_class):
            print("Decreto:", n[1])
            print("Link:",  decretos_df.loc[n[1]][3])
            print("Presidente:", decretos_df.loc[n[1]][2])
            print("Referenda:", decretos_df.loc[n[1]][4])
            print("Classificação Senado:", to_names(decretos_minist_senado.loc[n[1]]['classificação senado'], classes_sen_names))
            print("Classificação Referenda:", to_names(decretos_minist_senado.loc[n[1]]['referenda'], classes_minist_names))
            print("\nEmenta:\n",decretos_df.loc[n[1]][1])
            print("\n---------------------------------------------------------------------------------------------\n")

            full_list_referenda.append(decretos_minist_senado.loc[n[1]]['referenda'])
            full_list_sen.append(decretos_minist_senado.loc[n[1]]['classificação senado'])
            counter+=1

    print("Senado Report:")      
    full_list_sen = np.array(full_list_sen).sum(0) / counter 
    for i in range(len(full_list_sen)):
        print(classes_sen_names[i]+":", round(full_list_sen[i]*100, 2))
    print("\n")

    print("Referenda Report:")
    full_list_referenda = np.array(full_list_referenda).sum(0) / counter 
    for i in range(len(full_list_referenda)):
        print(classes_minist_names[i]+":", round(full_list_referenda[i]*100, 2))

In [9]:
modularity_clusters = {}
for na in nodes_analysis[1:]:
    if na[6] not in modularity_clusters.keys():
        modularity_clusters[na[6]] = 1
    else:
        modularity_clusters[na[6]] += 1

min_size = int(len(nodes_analysis)*0.01)
for k in list(modularity_clusters.keys()):
    if modularity_clusters[k] < min_size:
        modularity_clusters.pop(k)
    
modularity_clusters = set(modularity_clusters.keys())

In [10]:

default_output = sys.stdout
for mc in modularity_clusters:
    sys.stdout = open("modularity_analysis_v2/"+mc+".txt", "w", encoding='UTF-8')
    print_modularity_analysis(mc)
    sys.stdout = default_output

### Main modularity clusters relations

In [11]:
nodes_dict = {}
for na in nodes_analysis[1:]:
    nodes_dict[na[0]] = na[6]

In [12]:
with open('graph csv/edges_gephi_v3.csv', newline='') as csvfile:
    edges_analysis = list(csv.reader(csvfile))

edges_dict = {}
for ea in edges_analysis[1:]:
    edges_dict[(ea[0], ea[1])] = (nodes_dict[ea[0]])

In [13]:
edges_modularity = {}
for e in edges_dict:
    if edges_dict[e] != nodes_dict[e[1]]:
        if (edges_dict[e], nodes_dict[e[1]]) not in edges_modularity:
            edges_modularity[(edges_dict[e], nodes_dict[e[1]])] = 1
        else:
            edges_modularity[(edges_dict[e], nodes_dict[e[1]])] += 1


In [14]:
for em in list(edges_modularity.keys()):
    if em[0] not in modularity_clusters or em[1] not in modularity_clusters:
        edges_modularity.pop((em[0], em[1]))

In [15]:
len(edges_modularity)

184

In [16]:
source = []
target = []
weights = []
for st in edges_modularity:
    source.append(st[0])
    target.append(st[1])
    weights.append(edges_modularity[st])
edges_gephi = pd.DataFrame()
edges_gephi["Source"] = source
edges_gephi["Target"] = target
edges_gephi["Label"] = weights
edges_gephi.to_csv("modularity_analysis_v2/edges_gephi.csv", index=False)
edges_gephi

,Source,Target,Label
0,0,363,1
1,0,131,2
2,0,8,1
3,0,177,2
4,0,54,3
...,...,...,...
179,356,54,2
180,8,71,3
181,333,349,1
182,8,349,1


In [17]:
nodes_gephi = pd.DataFrame()
nodes_gephi["Id"] = list(modularity_clusters)
nodes_gephi["Label"] = list(modularity_clusters)
nodes_gephi.to_csv("modularity_analysis_v2/nodes_gephi.csv", index=False)
nodes_gephi

,Id,Label
0,316,316
1,131,131
2,356,356
3,363,363
4,75,75
5,108,108
6,132,132
7,85,85
8,333,333
9,102,102
